### Import Required Libraries and Set Up Environment Variables

In [44]:
!pip install python-dotenv
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [45]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [46]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# query_url = url + "api-key=" + nyt_api_key + "&q=" + filter_query
# query_url = f"{url}api-key={nyt_api_key}&q={filter_query}&begin_date={begin_date}&end_date={end_date}"
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [47]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
# create query with a page number
# API results show 10 articles at a time
for page in range(0, 20):
    
    # extend query with page number
    query_url_page = f"{query_url}&page={str(page)}"
     
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url_page).json()
    # print(reviews)
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    # loop through the reviews["response"]["docs"] and append each review to the list
    for review in reviews["response"]["docs"]:
        try:
            reviews_list.append(review)
            print(f"{review['headline']['print_headline']} found! Appending review. Page {page}")
        
        # Handle exceptions for a reviews that are not available in the NYT API
        except:
            # Print exception note
            print(f"No reviews found on page {page}.")
            break

The Attachment Diaries found! Appending review. Page 0
What’s Love Got to Do With It? found! Appending review. Page 0
You Can Live Forever found! Appending review. Page 0
A Tourist’s  Guide to Love found! Appending review. Page 0
Intoxicating Love With a Sobering Turn found! Appending review. Page 0
One True Loves found! Appending review. Page 0
The Lost Weekend:  A Love Story found! Appending review. Page 0
An Unbending Will Meets a Shifting City found! Appending review. Page 0
They Have a Humdrum Kind of Love found! Appending review. Page 0
Love in the Time Of Fentanyl found! Appending review. Page 0
Pamela, a Love Story found! Appending review. Page 1
In From the Side found! Appending review. Page 1
After Love found! Appending review. Page 1
Alcarràs found! Appending review. Page 1
Nelly &amp; Nadine found! Appending review. Page 1
A Love Whose Name Is Often Spoken found! Appending review. Page 1
The Sound  Of Christmas found! Appending review. Page 1
A Few Good Men, Some With Secre

In [48]:
# Preview the first 5 results in JSON format
json_string = json.dumps(reviews_list[:5], indent=4)
print(json_string)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [49]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_results_df = pd.json_normalize(reviews_list)
nyt_results_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [50]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_results_df['title'] = nyt_results_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
nyt_results_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [51]:
# defined function to extract key words
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # print(keyword)
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Extract 'name' and 'value' from items in "keywords" column

results_keywords_df = nyt_results_df['keywords'].apply(extract_keywords)
results_keywords_df = pd.DataFrame(results_keywords_df, columns=['keywords'])
# print(results_keywords_df)

# Fix the "keywords" column by converting cells from a list to a string
results_keywords_df['keywords'] = results_keywords_df['keywords'].astype(str)
results_keywords_df['keywords']

# add the column to the df
nyt_results_df['keywords'] = results_keywords_df['keywords']

In [52]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = nyt_results_df['title'].to_list()
print(title_list)

['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

### Access The Movie Database API

In [53]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the reviews
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
sleep_counter = 50
request_counter = 1
num_sleeps = 0

# Loop through the titles
for titles in title_list:

    # Check if we need to sleep before making a request
    if (request_counter % sleep_counter) == 0:
        time.sleep(1)
        num_sleeps += 1
        print(f"Sleep interval #: {num_sleeps}")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database based on titles
    tmdb_result = requests.get(url + titles + tmdb_key_string).json()

    # Try to extract data out of the API results
    try:
        # Extract movie ID
        movie_id = tmdb_result["results"][0]['id']

        # Create new URL for second API search based on movie ID
        url2 = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Perform a "GET" request for The Movie Database based on movie ID
        tmdb2_api_results = requests.get(url2).json()
       
        # Extract genre names into list
        genres = []
        for genre in tmdb2_api_results['genres']:
            genres.append(genre['name'])

        # Extract the spoken languages English name
        spoken_languages = []
        for languages in tmdb2_api_results['spoken_languages']:
            spoken_languages.append(languages['english_name'])

        # Extract the production countries
        production_countries = []
        for countries in tmdb2_api_results['production_countries']:
            production_countries.append(countries['name'])

        # Extract values from the API results into the movie list 
        tmdb_movies_list.append({
            'title' : tmdb2_api_results['title'],
            'original_title' : tmdb2_api_results['original_title'],
            'budget' : tmdb2_api_results['budget'],
            'original_language' : tmdb2_api_results['original_language'],
            'homepage' : tmdb2_api_results['homepage'],
            'overview' : tmdb2_api_results['overview'],
            'popularity' : tmdb2_api_results['popularity'],
            'runtime' : tmdb2_api_results['runtime'],
            'revenue' : tmdb2_api_results['revenue'],
            'release_date' : tmdb2_api_results['release_date'],
            'vote_average' : tmdb2_api_results['vote_average'],
            'vote_count' : tmdb2_api_results['vote_count'],
            'generes' : genres,
            'spoken_language' : spoken_languages,
            'production_countries' : production_countries})

        # For any successful attempts on the API result extractions:
        print(f"{titles} was found.")
                
    # For any failed attempts on the API result extractions:
    except:
        print(f"{titles} NOT found.")

The Attachment Diaries was found.
What’s Love Got to Do With It?’ Probably a Lo NOT found.
You Can Live Forever was found.
A Tourist’s Guide to Love was found.
Other People’s Children was found.
One True Loves was found.
The Lost Weekend: A Love Story was found.
A Thousand and One was found.
Your Place or Mine was found.
Love in the Time of Fentanyl was found.
Pamela, a Love Story was found.
In From the Side was found.
After Love was found.
Alcarràs was found.
Nelly & Nadine was found.
Lady Chatterley’s Lover was found.
The Sound of Christmas was found.
The Inspection was found.
Bones and All was found.
My Policeman was found.
About Fate was found.
Waiting for Bojangles was found.
I Love My Dad was found.
A Love Song was found.
Alone Together was found.
Art of Love was found.
The Wheel was found.
Thor: Love and Thunder was found.
Both Sides of the Blade was found.
Fire of Love was found.
Love & Gelato was found.
Stay Prayed Up was found.
Benediction was found.
Dinner in America was fou

In [ ]:
# Preview the first 5 results in JSON format
print(json.dumps(tmdb_movies_list[:5], indent =4))

In [ ]:
# Convert the results to a dataframe
tmdb_df = pd.DataFrame(tmdb_movies_list)

# Print the dataframe
tmdb_df

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merge_df = pd.merge(nyt_results_df, tmdb_df, how='left', on=["title"])
print(merge_df.columns)
merge_df.head(5)

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Define a function to check if a column contains lists
def column_contains_list(cols):
    return any(isinstance(val, list) for val in cols)

# Identify columns containing lists
columns_to_fix = [cols for cols in merge_df.columns if column_contains_list(merge_df[cols])]
print(columns_to_fix)

# Create a list of characters to remove
characters_to_remove = ["[","]", "{", "}","'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    
    # Convert the column to type 'str'
    merge_df[col] = merge_df[col].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merge_df[col] = merge_df[col].str.replace(char, '')

# Display the fixed DataFrame
merge_df.head()

In [ ]:
# Drop "byline.person" column
merge_df.drop('byline.person', axis=1, inplace=True)
merge_df.columns

In [ ]:
# Delete duplicate rows and reset index
old_length = len(merge_df)
merge_df = merge_df.drop_duplicates()
num_dups = old_length - len(merge_df)
if num_dups > 0:
    print(f"{num_dups} duplicate rows were deleted.")
    merge_df = merge_df.reset_index()
else:
    print("There were no duplicate rows to delete.")

# Export data to CSV without the index
census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)

In [ ]:
# Save the DataFrame as a CSV
merge_df.to_csv("nyt_tmdb_merged_movie_data.csv", encoding="utf-8", index=False)